# TFIDF

TFIDF Embedding can not be used for vectorizing the vector for finfing the similarity with the query as it generates its courpus everytime we create TFIDF matrix which requried huge calculation to be performed everytime we need to perform similarity calculation.

## Word2Vec

In [123]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

In [56]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=ada8cc8b98fd8643ced5f03068bec79f0a4728c1c4c9ad7f7975f9f5177a98ad
  Stored in directory: c:\users\dhrsingh\appdata\local\pip\cache\wheels\04\5f\3e\46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


### Importing weights for embedding Google Pretrained Model

In [124]:
filepath = "GoogleNews-vectors-negative300.bin.gz"

from gensim.models import KeyedVectors
wv_from_bin = KeyedVectors.load_word2vec_format(filepath, binary=True) 
#extracting words7 vectors from google news vector
embeddings_index = {}

In [125]:
for word, vector in zip(wv_from_bin.key_to_index , wv_from_bin.vectors):
    coefs = np.asarray(vector, dtype='float32')
    embeddings_index[word] = coefs

In [132]:
## fucntion to findout the vector space of each sentence

def avg_feature_vector(sentence, model, num_features):
    words = sentence.split()
    #feature vector is initialized as an empty array
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in embeddings_index.keys():
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [133]:
import pandas as pd
import numpy as np

df = pd.read_csv("stratified_sample_1.csv", usecols= ['clean_detail_desc', 'prod_name'])

In [134]:
df.head()

,prod_name,clean_detail_desc
0,Vickan dress w,short dress airi chiffon flouncetrim vneck sew...
1,Vickan dress w,short dress airi chiffon flouncetrim vneck sew...
2,Vickan dress w,short dress airi chiffon flouncetrim vneck sew...
3,Vickan dress w,short dress airi chiffon flouncetrim vneck sew...
4,Vickan dress w,short dress airi chiffon flouncetrim vneck sew...


In [231]:
import random
res = random.sample(range(1, 1589416), 10)
clean_data = df.loc[res].clean_detail_desc.tolist()
prod_name = df.loc[res].prod_name.tolist()

In [232]:
df.loc[res]

,prod_name,clean_detail_desc
1116957,Seb overshirt,overs shirt jacket wash cotton denim collar bu...
81117,Jan tee,tshirt pattern cotton jersey round necklin che...
760759,Orlando padded soft bra,soft nonwir bra lace adjust strap cup remov in...
943975,Luna skinny 5 pkt,pocket jean wash superstretch denim regular wa...
979649,ZEBRA CF TVP,top lightweight sweatshirt fabric drop shoulde...
401968,CALLISTO ski jkt,pad ski jacket windproof waterrepel breathabl ...
346390,ESSENTIAL FIONA LACE TANK,fit top soft rib cotton jersey vneck lace trim...
465349,Simple as That Triangle Top,line nonwir triangl bikini top wide hem narrow...
1143228,Ozzy Denim Shorts,highwaist short superstretch twill back pocket...
1015978,Shorts HW Pixie,highwaist short wash stretch denim zip fli but...


In [239]:
query = ["Denim Jacket", "Rounk Neck Tshirt" , "padded soft bra", "denim jeans", "Sweatshirt", "Windproof Jacket", "Top" , "triangl bikini","denim shorts", "shorts"]

In [235]:
## this function measure the spatical distace between product description and product name. 
## the smaller the distance the more similar is the graph
sim = []
for i,j in zip(clean_data , prod_name):
    s1_afv = avg_feature_vector(i , model= embeddings_index, num_features=300 )
    s2_afv = avg_feature_vector(j , model= embeddings_index, num_features=300 )
    cos = distance.cosine(s1_afv, s2_afv)
    print(cos)
    sim.append(cos)


    

0.6370903849601746
0.7032435536384583
0.37777626514434814
0.5852794051170349
0.9826638586819172
0.6709148585796356
0.912772923707962
0.6733211278915405
0.7004756927490234
0.742926687002182


In [240]:
## this function measure the spatical distace between product description and product name. 
## the smaller the distance the more similar is the graph
query_sim = []
for i,j in zip(clean_data , query):
    s1_afv = avg_feature_vector(i , model= embeddings_index, num_features=300 )
    s2_afv = avg_feature_vector(j , model= embeddings_index, num_features=300 )
    cos = distance.cosine(s1_afv, s2_afv)
    print(cos)
    query_sim.append(cos)


    

0.6336083114147186
0.5499168932437897
0.31116408109664917
0.3403424620628357
0.68471759557724
0.591669112443924
0.7376987040042877
0.6035303473472595
0.42207491397857666
0.5469109416007996


# Bert Algorithm

### Simple Bert

In [242]:
from scipy.spatial import distance

In [3]:
pip install -U sentence-transformers


  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120751 sha256=c4254dfdd1b37b33018f44d962fc4858ad7b2fc8b26fabde070d043e7b31cc6c
  Stored in directory: c:\users\dhrsingh\appdata\local\pip\cache\wheels\2b\11\3b\32a18fb9f2253b25d3d1a06f0a84e2d516e7efa19c8c71a283
Successfully built sentence-transformers
Note: you may need to restart the kernel to use updated packages.


In [243]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [244]:
clean_data_sentence_embeddings = model.encode(clean_data)
prod_name_embeddings = model.encode(prod_name)

In [245]:
clean_data_sentence_embeddings.shape

(10, 768)

In [246]:
prod_name_embeddings.shape

(10, 768)

In [156]:
from sklearn.metrics.pairwise import cosine_similarity

In [247]:
## this function measure the spatical distace between product description and product name. 
## the smaller the distance the more similar is the graph
sbert_sim = []
for i,j in zip(clean_data_sentence_embeddings, prod_name_embeddings):
    s1_afv = i
    s2_afv = j
    cos = distance.cosine(s1_afv, s2_afv)
    print(cos)
    sbert_sim.append(cos)


    

0.5291747450828552
0.774872362613678
0.38676708936691284
0.706072986125946
0.673152506351471
0.5088545083999634
0.2938309907913208
0.5849405825138092
0.33392322063446045
0.4877750277519226


In [248]:
## this function measure the spatical distace between product description and product name. 
## the smaller the distance the more similar is the graph
query_sbert_sim = []
for i,j in zip(clean_data_sentence_embeddings, model.encode(query)):
    s1_afv = i
    s2_afv = j
    cos = distance.cosine(s1_afv, s2_afv)
    print(cos)
    query_sbert_sim.append(cos)


    

0.48398202657699585
0.33558523654937744
0.37731319665908813
0.387798547744751
0.389896035194397
0.6278940141201019
0.6838354766368866
0.2952868938446045
0.38006848096847534
0.5320334434509277


# Bert Algorithm

In [249]:
from transformers import AutoTokenizer, AutoModel
import torch

In [173]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')


In [250]:
def BERT(sentences):
    
    
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
    model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
    
    # initialize dictionary to store tokenized sentences
    tokens = {'input_ids': [], 'attention_mask': []}

    for sentence in sentences:
        # encode each sentence and append to dictionary
        new_tokens = tokenizer.encode_plus(sentence, max_length=500,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
        tokens['input_ids'].append(new_tokens['input_ids'][0])
        tokens['attention_mask'].append(new_tokens['attention_mask'][0])

        

    # reformat list of tensors into single tensor
    tokens['input_ids'] = torch.stack(tokens['input_ids'])
    tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
    
    outputs = model(**tokens)
    outputs.keys()
    
    embeddings = outputs.last_hidden_state
    embeddings
    
    embeddings.shape
    
    attention_mask = tokens['attention_mask']
    attention_mask.shape
    
    mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    mask.shape

    
    masked_embeddings = embeddings * mask
    masked_embeddings
    
    summed = torch.sum(masked_embeddings, 1)
    summed.shape
    
    summed_mask = torch.clamp(mask.sum(1), min=1e-9)
    summed_mask.shape
    
    mean_pooled = summed / summed_mask
    
    return mean_pooled
    
    
    
    
    
    

In [214]:
import cleantext
clean_prod_name = []
for i in prod_name:
    clean_prod_name.append(cleantext.clean(i)) 

In [215]:
prod_name = clean_prod_name

In [222]:
prod_name

['penni strap top',
 'emi linen short',
 'ned nasa',
 'essenti sign boat neck',
 'brit babi tee',
 'new pluto dress',
 'carri p brazilian',
 'kelli strapless sofia pk',
 'primo slack',
 'bristol']

In [223]:
description_embedding.shape

(10, 768)

In [255]:
query_name_embedding = []
for i in query:
    print(i)
    query_name_embedding.append(BERT([i]).detach().numpy()[0])
    print()
    

Denim Jacket

Rounk Neck Tshirt

padded soft bra

denim jeans

Sweatshirt

Windproof Jacket

Top

triangl bikini

denim shorts

shorts



In [254]:
prod_name_embedding = []
for i in prod_name:
    print(i)
    prod_name_embedding.append(BERT([i]).detach().numpy()[0])
    print()
    

Seb overshirt

Jan tee

Orlando padded soft bra

Luna skinny 5 pkt

ZEBRA CF TVP

CALLISTO ski jkt

ESSENTIAL FIONA LACE TANK

Simple as That Triangle Top

Ozzy Denim Shorts

Shorts HW Pixie



In [251]:
description_embedding = BERT("clean_data").detach().numpy()

In [216]:
prod_name_embedding = BERT("prod_name").detach().numpy()

In [252]:
query_name_embedding = BERT("query").detach().numpy()

In [256]:
## this function measure the spatical distace between product description and product name. 
## the smaller the distance the more similar is the graph
Bert_sim = []
j = 0
for i,j in zip(description_embedding, prod_name_embedding):
    s1_afv = i
    s2_afv = j
    cos = distance.cosine(s1_afv, s2_afv)
    print(cos)
    Bert_sim.append(cos)
    


    

0.39815521240234375
0.31475579738616943
0.6040824353694916
0.5813363194465637
0.39055687189102173
0.567454606294632
0.5486803948879242
0.4145025610923767
0.5293334722518921
0.5735144317150116


In [257]:
## this function measure the spatical distace between product description and product name. 
## the smaller the distance the more similar is the graph
query_Bert_sim = []
j = 0
for i,j in zip(description_embedding, query_name_embedding):
    s1_afv = i
    s2_afv = j
    cos = distance.cosine(s1_afv, s2_afv)
    print(cos)
    query_Bert_sim.append(cos)
    


    

0.5102702081203461
0.47770261764526367
0.5390638113021851
0.5125288665294647
0.27759867906570435
0.5872719585895538
0.15944403409957886
0.49699848890304565
0.5027430951595306
0.2733139991760254


# FuzzyWuzzy Similarity

In [183]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [185]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.Collecting python-Levenshtein



  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\dhrsingh\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\dhrsingh\\AppData\\Local\\Temp\\pip-install-lm5xls0t\\python-levenshtein_18486c8811cf47429b4a2e1f1f18aff3\\setup.py'"'"'; __file__='"'"'C:\\Users\\dhrsingh\\AppData\\Local\\Temp\\pip-install-lm5xls0t\\python-levenshtein_18486c8811cf47429b4a2e1f1f18aff3\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\dhrsingh\AppData\Local\Temp\pip-wheel-x__d1ox9'
       cwd: C:\Users\dhrsingh\AppData\Local\Temp\pip-install-lm5xls0t\python-levenshtein_18486c8811cf47429b4a2e1f1f18aff3\
  Complete output (28 lines):
  running bdist_wheel
  running build
  running build_py
  creatin

  Running setup.py clean for python-Levenshtein
Failed to build python-Levenshtein
    Running setup.py install for python-Levenshtein: started
    Running setup.py install for python-Levenshtein: finished with status 'error'



  adding license file 'COPYING'
  writing manifest file 'python_Levenshtein.egg-info\SOURCES.txt'
  copying Levenshtein\_levenshtein.c -> build\lib.win-amd64-3.9\Levenshtein
  copying Levenshtein\_levenshtein.h -> build\lib.win-amd64-3.9\Levenshtein
  running build_ext
  building 'Levenshtein._levenshtein' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  ----------------------------------------
  ERROR: Failed building wheel for python-Levenshtein
    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\dhrsingh\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\dhrsingh\\AppData\\Local\\Temp\\pip-install-lm5xls0t\\python-levenshtein_18486c8811cf47429b4a2e1f1f18aff3\\setup.py'"'"'; __file__='"'"'C:\\Users\\dhrsingh\\AppData\\Local\\Temp\\pip-install-lm5xls0t\\python-levenshtein_18486c8811cf47429b4a

In [186]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [258]:
fuzzywuzzy_sim = []
query_fuzzywuzzy_sim = []

for i,j,k in zip(clean_data, prod_name, query):
    fuzzywuzzy_sim.append(fuzz.ratio( i , j )*0.01)
    query_fuzzywuzzy_sim.append(fuzz.ratio( i , k )*0.01)



In [192]:
fuzzywuzzy_sim

[0.17, 0.17, 0.04, 0.06, 0.32, 0.1, 0.15, 0.18, 0.16, 0.09]

In [259]:
query_fuzzywuzzy_sim

[0.09, 0.14, 0.12, 0.21, 0.21, 0.08, 0.05, 0.18, 0.21, 0.11]

In [262]:
result = df.loc[res]

In [263]:
result["Query"] = query
result["Word2Vec Score"] = sim
result["Simple Bert Score"] = sbert_sim
result["Bert Score"] = Bert_sim
result["FuzzyWuzzy Score"] = fuzzywuzzy_sim
result["Query Word2Vec Score"] = query_sim
result["Query Simple Bert Score"] = query_sbert_sim
result["Query Bert Score"] = query_Bert_sim
result["Query FuzzyWuzzy Score"] = query_fuzzywuzzy_sim



In [264]:
result

,prod_name,clean_detail_desc,Query,Word2Vec Score,Simple Bert Score,Bert Score,FuzzyWuzzy Score,Query Word2Vec Score,Query Simple Bert Score,Query Bert Score,Query FuzzyWuzzy Score
1116957,Seb overshirt,overs shirt jacket wash cotton denim collar bu...,Denim Jacket,0.637090,0.529175,0.398155,0.11,0.633608,0.483982,0.510270,0.09
81117,Jan tee,tshirt pattern cotton jersey round necklin che...,Rounk Neck Tshirt,0.703244,0.774872,0.314756,0.13,0.549917,0.335585,0.477703,0.14
760759,Orlando padded soft bra,soft nonwir bra lace adjust strap cup remov in...,padded soft bra,0.377776,0.386767,0.604082,0.12,0.311164,0.377313,0.539064,0.12
943975,Luna skinny 5 pkt,pocket jean wash superstretch denim regular wa...,denim jeans,0.585279,0.706073,0.581336,0.17,0.340342,0.387799,0.512529,0.21
979649,ZEBRA CF TVP,top lightweight sweatshirt fabric drop shoulde...,Sweatshirt,0.982664,0.673153,0.390557,0.05,0.684718,0.389896,0.277599,0.21
401968,CALLISTO ski jkt,pad ski jacket windproof waterrepel breathabl ...,Windproof Jacket,0.670915,0.508855,0.567455,0.05,0.591669,0.627894,0.587272,0.08
346390,ESSENTIAL FIONA LACE TANK,fit top soft rib cotton jersey vneck lace trim...,Top,0.912773,0.293831,0.548680,0.06,0.737699,0.683835,0.159444,0.05
465349,Simple as That Triangle Top,line nonwir triangl bikini top wide hem narrow...,triangl bikini,0.673321,0.584941,0.414503,0.16,0.603530,0.295287,0.496998,0.18
1143228,Ozzy Denim Shorts,highwaist short superstretch twill back pocket...,denim shorts,0.700476,0.333923,0.529333,0.17,0.422075,0.380068,0.502743,0.21
1015978,Shorts HW Pixie,highwaist short wash stretch denim zip fli but...,shorts,0.742927,0.487775,0.573514,0.12,0.546911,0.532033,0.273314,0.11


In [265]:
result.to_csv("Embedding Model.csv")